This Jupyter note book includes some codes to start WebScraping. We will use a package called "BEAUTIFUL SOUP" to collect data from web.
Once you've collected your data and stored it in .csv file you can start analysis.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 20
page_size = 100

In [3]:
# loading the required pages by getting requests from web

for t in range(1, pages + 1):

    print(f"Scraping page {t}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{t}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response=requests.get(url).content
    soup=BeautifulSoup(response,"lxml")
    review_box = soup.find_all("article",{"itemprop":"review"})

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20


In [4]:
# creating lists and dict for creating DF

date_li=[]
name_li=[]
rating_li=[]
verify_li=[]
review_li=[]

columns=[ 'Seat Type', 'Recommended','Aircraft', 'Type Of Traveller', 'Route']  
        
valued_col=['Ground Service', 'Wifi & Connectivity','Seat Comfort','Value For Money', 'Inflight Entertainment',
                'Food & Beverages', 'Cabin Staff Service']


dict2 = {'Aircraft':[], 'Seat Type':[],'Type Of Traveller':[] ,'Recommended':[] ,'Ground Service':[],
        'Wifi & Connectivity':[], 'Seat Comfort':[], 
        'Value For Money':[], 'Inflight Entertainment':[],
        'Food & Beverages':[], 'Cabin Staff Service':[],'Route':[]}

In [5]:
#finding Unique values in table columns

list=[]
for rev in review_box:
    table=rev.find("table",{"class":"review-ratings"})
    rows = table.find_all("tr")
    for row in rows:
        data=row.find_all("td")
        for x in data:
            list.append(x.get_text())

double_li = [list[x:x+2] for x in range(0,len(list),2)]





# to find column elements
l=[]
for x in double_li:
    t=x[0]
    l.append(t)     # appending table columns to a list "l"
s=set(l)            # getting unique values
# print(s)
# len(s)

In [6]:
# for t in range(1, pages + 1):
#     review_box = soup.find_all("article",{"itemprop":"review"})
#     for rev in review_box:
#         print(rev)


In [7]:
# getting published date

for t in range(1, pages + 1):
   
    for rev in review_box:
        date = rev.find("meta")["content"]
        if date == None:
            date_li.append("NA")
        elif date != None:
            date_li.append(date)
# print(date_li)    
# print(len(date_li))

In [8]:
for t in range(1, pages + 1):  #getting overall rating
    for rev in review_box:
        rating=rev.find("div",{"class":"rating-10"}).find("span",{"itemprop":"ratingValue"})
        if rating == None:
            rating_li.append("NA")
        elif rating !=None:
            rating_li.append(rating.text)
# print(rating_li)   
# print(len(rating_li))

In [9]:
for t in range(1, pages + 1): # getting name of customer
    for rev in review_box:
    
        names=rev.find("h3").find("span",{"itemprop":"author"}).find("span",{"itemprop":"name"})
        if names == None:
            name_li.append("NA")
        elif names != None:
            name_li.append(names.get_text())
        
# print(name_li)
# print(len(name_li))

In [10]:
 # checking and getting verification status of review

for t in range(1, pages + 1):   
    for rev in review_box:
        verify=rev.find("div",{"class":"tc_mobile"}).find("div",{"class":"text_content"}).find("a")
        if verify==None:
            verify_li.append("NA")
        elif verify != None:
            verify_li.append(verify.get_text())



# print(verify_li)
# print(len(verify_li))

In [11]:
# getting customer review

for t in range(1,pages+1):   
    for rev in review_box:
        review=rev.find("h2")
        if review == None:
            review_li.append("NA")
        elif review != None:
            review=review.get_text()
            review_li.append(review[1:-1])
# print(review_li)
# print(len(review_li))

In [12]:
# obtaining all rows of the table

row_data=[]
for t in range(1,pages+1):
    
    for rev in review_box:
        table=rev.find("table",{"class":"review-ratings"})
        rows = table.find_all("tr")
        row_data.append(rows)
#print(row_data)

In [13]:
# moving all table keys and values into a dictionary

for t in range(1,pages+1):

    j=0
    while j < 100:
        ele_li=[]
        
        for ele in row_data[j]:
            col = ele.find_all("td")
            ele_li.append(col)
        
        i=0


        while i < len(ele_li):
            


            for k in columns:
                dict2[k]
                if ele_li[i][0].get_text() == k:
                    dict_val=ele_li[i][1].get_text()
                    

                    dict2[k].append( dict_val)
        
            for k in valued_col:
                if ele_li[i][0].get_text() == k:

                    for q in ele_li[i]:
                        rate = q.find_all("span",{"class":"star fill"})
                        str=""
                        for x in rate:
                            str +=x.get_text()
                    dict2[k].append(str[-1])


            i +=1

        lens = dict2.keys()
        for item in lens:
            if len(dict2[item]) != len(dict2["Recommended"]):
                dict2[item].append("NA")
            
            
        j+=1


print("successful")

   

successful


In [14]:
lens = dict2.values()
for item in lens:
    print(len(item))

2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000


In [15]:
dict1={ "Rating":rating_li,
      "Review":review_li,
      "Verification":verify_li,
      "Customer":name_li,
      "Published Date":date_li}


In [16]:
# concatenating two dictionary

dict={**dict1,**dict2}
#dict

In [17]:
scrape=pd.DataFrame(dict)


In [19]:
scrape.to_csv("reviews_BA.csv",index=False)
print("complted")

complted
